In [ ]:
import xarray as xr
import rioxarray as rxr
from geoarray import GeoArray
from arosics import COREG
from arosics import COREG_LOCAL
import os
import re

In [ ]:
# Read in data
ref = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/sentinel_data/Sentinel-2_2021_Yamal_GS_Mosaic_bgrnir_UTM42N.tif'
tgt_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/planet_data/initial_data_download/files/'
tgt_files = [file for file in os.listdir(tgt_dir) if re.match('.*SR.*tif$', file)]
tgt_files.sort()
udm_files = [file for file in os.listdir(tgt_dir) if re.match('.*udm2.*tif$', file)]
udm_files.sort()
mask_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/planet_data/initial_data_download/masked_files/'
out_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/calibrated_composite/arosics_output/calibrated_files/'
out_fig_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/calibrated_composite/arosics_output/figures/'
print(len(tgt_files), len(udm_files))
tgt_files

In [ ]:
for tgt_file, udm_file in zip(tgt_files, udm_files):
    
    # Mask imagery using clear mask
    tgt_file_full = tgt_dir + tgt_file
    udm_file_full = tgt_dir + udm_file
    tgt = rxr.open_rasterio(tgt_file_full)
    udm = rxr.open_rasterio(udm_file_full).sel(band = 1)
    tgt = tgt.where(tgt != 0)
    tgt = tgt.where(udm != 0)
    tgt.rio.to_raster(mask_dir + tgt_file[0:-4] + '_masked.tif')
    
    # Apply AROSICS correction from red band to all bands
    tgt_file_full = mask_dir + tgt_file[0:-4] + '_masked.tif'
    out_path = out_dir + tgt_file[0:-4] + '_arosics.tif'
    out_fig_path = out_fig_dir + tgt_file[0:-4] + '_coreg_points.tif'
    kwargs = {
        'grid_res'     : 200, # can make this larger to improve run time
        'window_size'  : (200, 200), # A larger window size could slow things down, but too small might lead to less accurate alignment
        'q'            : False,
        'r_b4match'    : 3,
        's_b4match'    : 3,
        'path_out'     : out_path,
        'fmt_out'      : 'GTiff',
        'CPUs'         : 19
    }
    tgt_cal = COREG_LOCAL(ref, tgt_file_full, **kwargs)
    tgt_cal.correct_shifts(min_points_local_corr = 4)
    tgt_cal.view_CoRegPoints(savefigPath = out_fig_path)
    